In [9]:
import time
!pip install tweet-preprocessor
import pandas as pd
import preprocessor as p
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from googletrans import Translator
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tosin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tosin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tosin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [10]:
def data_download(filename):
    data = pd.read_table(filename, sep="\t",header=0).dropna()
    print(data.head())
    return data

In [11]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

# function for preprocessing tweet in preparation for sentiment analysis
def ProcessedTweets(text):
    #changing tweet text to small letters
    text = re.sub('(MT|mT|Mt)', '', text)  # remove re-tweet
    text = re.sub('(RT|rT|Rt)', '', text)
    text = text.lower()
    # Removing @ and links
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split())
    text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)  # remove tweeted at
    text = re.sub(r'\\n', '', text)
    # removing punctuation and numbers
    punct = str.maketrans('', '', string.punctuation)
    text = text.translate(punct)

    text = remove_emojis(text)
    # tokenizing words and removing stop words from the tweet text
    tokens = word_tokenize(text)
    # lemmetizing words
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w) for w in tokens]
    text = " ".join(lemma_words)
    return text



In [12]:
import string
translator = Translator()
translator.raise_Exception = True
def translate(text):
    time.sleep(0.5)
    return translator.translate(text,dest='en').text
def translate_data(data):
    data.tweetText = data.tweetText.apply(lambda x : translate(x))

In [13]:
def preprocess_data(data):
    data.tweetText = data.tweetText.apply(lambda x : ProcessedTweets(x))
    data.drop(columns=['username', 'timestamp','userId','imageId(s)'])
    data['label'] = data.label.apply(lambda  x : 'fake' if x == 'humor' else x )
    print(data.head())

In [14]:
#def download_train_data():
train_data = data_download('assignment-comp3222-comp6246-mediaeval2015-dataset/mediaeval-2015-trainingset.txt')
regex = re.compile('r^(RT|rT|Rt|rt)')
retweets = train_data.tweetText.apply(lambda x : bool(regex.search(str(x))))
train_data = train_data[~retweets]
translate_data(train_data)
preprocess_data(train_data)
train_data.to_csv('train.csv')

              tweetId                                          tweetText  \
0  263046056240115712  ¿Se acuerdan de la película: “El día después d...   
1  262995061304852481  @milenagimon: Miren a Sandy en NY!  Tremenda i...   
2  262979898002534400  Buena la foto del Huracán Sandy, me recuerda a...   
3  262996108400271360     Scary shit #hurricane #NY http://t.co/e4JLBUfH   
4  263018881839411200  My fave place in the world #nyc #hurricane #sa...   

      userId      imageId(s)        username                       timestamp  \
0   21226711  sandyA_fake_46         iAnnieM  Mon Oct 29 22:34:01 +0000 2012   
1  192378571  sandyA_fake_09  CarlosVerareal  Mon Oct 29 19:11:23 +0000 2012   
2  132303095  sandyA_fake_09     LucasPalape  Mon Oct 29 18:11:08 +0000 2012   
3  241995902  sandyA_fake_29     Haaaaarryyy  Mon Oct 29 19:15:33 +0000 2012   
4  250315890  sandyA_fake_15  princess__natt  Mon Oct 29 20:46:02 +0000 2012   

  label  
0  fake  
1  fake  
2  fake  
3  fake  
4  fake  


ConnectTimeout: _ssl.c:1112: The handshake operation timed out

In [ ]:
def download_test_data():
    train_data = data_download('assignment-comp3222-comp6246-mediaeval2015-dataset/mediaeval-2015-testset.txt')
    train_data = train_data[~retweets]
    translate_data(train_data)
    preprocess_data(train_data)
    train_data.to_csv('test.csv')

In [ ]:
download_test_data()